In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import Row
spark = SparkSession \
    .builder \
    .appName("Inferred and explicit schemas") \
    .getOrCreate()

In [2]:
lines = sc.textFile("./datasets/students.txt")
lines.collect()

['Emily,44,55,78', 'Andy,47,34,89', 'Rick,55,78,55', 'Aaron,66,34,98']

In [3]:
parts = lines.map(lambda l: l.split(","))
parts.collect()

[['Emily', '44', '55', '78'],
 ['Andy', '47', '34', '89'],
 ['Rick', '55', '78', '55'],
 ['Aaron', '66', '34', '98']]

In [4]:
# Convert every record to a Row object
students = parts.map(lambda p: Row(name=p[0], math=int(p[1]), english=int(p[2]), science=int(p[3])))

In [5]:
schemaStudents = spark.createDataFrame(students)

schemaStudents.createOrReplaceTempView("students")

In [6]:
# The created dataframe has column names, which is specified in the Row Object
schemaStudents.columns

['english', 'math', 'name', 'science']

#### Schema inferred based on the values in each column

In [7]:
# Any Schema for the dataframe is not explicitly specified
schemaStudents.schema

StructType(List(StructField(english,LongType,true),StructField(math,LongType,true),StructField(name,StringType,true),StructField(science,LongType,true)))

In [8]:
spark.sql("SELECT * FROM students").show()

+-------+----+-----+-------+
|english|math| name|science|
+-------+----+-----+-------+
|     55|  44|Emily|     78|
|     34|  47| Andy|     89|
|     78|  55| Rick|     55|
|     34|  66|Aaron|     98|
+-------+----+-----+-------+



#### Specifying Schema Explicitly

In [9]:
parts.collect()

[['Emily', '44', '55', '78'],
 ['Andy', '47', '34', '89'],
 ['Rick', '55', '78', '55'],
 ['Aaron', '66', '34', '98']]

In [10]:
schemaString = "name math english science"

In [12]:
from pyspark.sql.types import StructType, StructField, StringType, LongType

fields = [StructField('name', StringType(), True),
          StructField('math', LongType(), True),
          StructField('english', LongType(), True),
          StructField('science', LongType(), True),
]
schema = StructType(fields)

In [13]:
# Explicitly Specify the schema
schemaStudents = spark.createDataFrame(parts, schema)

In [14]:
schemaStudents.schema

StructType(List(StructField(name,StringType,true),StructField(math,LongType,true),StructField(english,LongType,true),StructField(science,LongType,true)))

In [15]:
spark.sql("SELECT * FROM students").show()

+-------+----+-----+-------+
|english|math| name|science|
+-------+----+-----+-------+
|     55|  44|Emily|     78|
|     34|  47| Andy|     89|
|     78|  55| Rick|     55|
|     34|  66|Aaron|     98|
+-------+----+-----+-------+

